<a href="https://colab.research.google.com/github/rezPirayesh/Customer-prediction/blob/main/Customer_Prediction_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
print("Hello, Colab works!")
import torch
print(torch.__version__)
print("GPU available:", torch.cuda.is_available())

Hello, Colab works!
2.6.0+cu124
GPU available: True


In [18]:
import pandas as pd
df = pd.read_csv('data.csv', encoding='ISO-8859-1')
print(df.head())  # First 5 rows
print(df.info())  # Columns, types
print(df['CustomerID'].notnull().value_counts())  # Purchase indicator

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo   

In [19]:
import torch
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(2, 4),
    nn.ReLU(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)
print("Model created:", model)

Model created: Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4, out_features=1, bias=True)
  (3): Sigmoid()
)


In [20]:
import pandas as pd
df = pd.read_csv('data.csv', encoding='ISO-8859-1')
df = df.dropna(subset=['CustomerID'])
df = df[df['Quantity'] > 0]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print("Cleaned dataset:")
print(df.info())
print(df.head())

Cleaned dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB
None
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3  

In [21]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
customer_data = df.groupby('CustomerID').agg({
    'TotalPrice': 'sum',
    'InvoiceNo': 'nunique',
    'Quantity': 'sum'
}).reset_index()
customer_data.columns = ['CustomerID', 'TotalSpent', 'PurchaseFrequency', 'TotalItems']
customer_data['Purchased'] = 1
print("Customer features:")
print(customer_data.head())
print(customer_data.info())

Customer features:
   CustomerID  TotalSpent  PurchaseFrequency  TotalItems  Purchased
0     12346.0    77183.60                  1       74215          1
1     12347.0     4310.00                  7        2458          1
2     12348.0     1797.24                  4        2341          1
3     12349.0     1757.55                  1         631          1
4     12350.0      334.40                  1         197          1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         4339 non-null   float64
 1   TotalSpent         4339 non-null   float64
 2   PurchaseFrequency  4339 non-null   int64  
 3   TotalItems         4339 non-null   int64  
 4   Purchased          4339 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 169.6 KB
None


In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch

# Features and target
X = customer_data[['TotalSpent', 'PurchaseFrequency', 'TotalItems']].values
y = customer_data['Purchased'].values

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train).to('cuda')
X_test = torch.FloatTensor(X_test).to('cuda')
y_train = torch.FloatTensor(y_train).to('cuda').reshape(-1, 1)
y_test = torch.FloatTensor(y_test).to('cuda').reshape(-1, 1)

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Training data shape: torch.Size([3471, 3])
Test data shape: torch.Size([868, 3])


In [23]:
print("Customer data size:", customer_data.shape)
print(customer_data.head())

Customer data size: (4339, 5)
   CustomerID  TotalSpent  PurchaseFrequency  TotalItems  Purchased
0     12346.0    77183.60                  1       74215          1
1     12347.0     4310.00                  7        2458          1
2     12348.0     1797.24                  4        2341          1
3     12349.0     1757.55                  1         631          1
4     12350.0      334.40                  1         197          1


In [24]:
import torch
import torch.nn as nn

class CustomerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(3, 16),  # 3 features
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

model = CustomerNet().to('cuda')
print(model)

CustomerNet(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [25]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 10, Loss: 0.8580
Epoch 20, Loss: 0.8278
Epoch 30, Loss: 0.8064
Epoch 40, Loss: 0.7873
Epoch 50, Loss: 0.7675
Epoch 60, Loss: 0.7412
Epoch 70, Loss: 0.7029
Epoch 80, Loss: 0.6533
Epoch 90, Loss: 0.5978
Epoch 100, Loss: 0.5383


In [26]:
import pandas as pd
df = pd.read_csv('data.csv', encoding='ISO-8859-1')
df = df.dropna(subset=['CustomerID'])
df = df[df['Quantity'] > 0]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
customer_data = df.groupby('CustomerID').agg({
    'TotalPrice': 'sum',
    'InvoiceNo': 'nunique',
    'Quantity': 'sum'
}).reset_index()
customer_data.columns = ['CustomerID', 'TotalSpent', 'PurchaseFrequency', 'TotalItems']
customer_data['Purchased'] = 1
print("Customer data size:", customer_data.shape)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
X = customer_data[['TotalSpent', 'PurchaseFrequency', 'TotalItems']].values
y = customer_data['Purchased'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train = torch.FloatTensor(X_train).to('cuda')
X_test = torch.FloatTensor(X_test).to('cuda')
y_train = torch.FloatTensor(y_train).to('cuda').reshape(-1, 1)
y_test = torch.FloatTensor(y_test).to('cuda').reshape(-1, 1)
print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Customer data size: (4339, 5)
Training data shape: torch.Size([3471, 3])
Test data shape: torch.Size([868, 3])


In [27]:
import torch
import torch.nn as nn
class CustomerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)
model = CustomerNet().to('cuda')
print(model)

import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

CustomerNet(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)
Epoch 10, Loss: 0.8411
Epoch 20, Loss: 0.8129
Epoch 30, Loss: 0.7836
Epoch 40, Loss: 0.7532
Epoch 50, Loss: 0.7206
Epoch 60, Loss: 0.6819
Epoch 70, Loss: 0.6355
Epoch 80, Loss: 0.5843
Epoch 90, Loss: 0.5310
Epoch 100, Loss: 0.4770


In [28]:
with torch.no_grad():
    model.eval()
    predictions = model(X_test)
    predictions = (predictions > 0.5).float()
    accuracy = (predictions == y_test).float().mean()
    print(f"Test Accuracy: {accuracy.item():.4f}")

Test Accuracy: 1.0000


In [29]:
import pandas as pd
import numpy as np
non_purchasers = pd.DataFrame({
    'CustomerID': range(999000, 999500),  # Unique IDs
    'TotalSpent': np.random.uniform(0, 50, 500),  # Low spending
    'PurchaseFrequency': np.ones(500),
    'TotalItems': np.random.randint(1, 5, 500),
    'Purchased': np.zeros(500)
})
customer_data = pd.concat([customer_data, non_purchasers], ignore_index=True)
print("Updated customer data size:", customer_data.shape)
print("Purchased distribution:", customer_data['Purchased'].value_counts())

Updated customer data size: (4839, 5)
Purchased distribution: Purchased
1.0    4339
0.0     500
Name: count, dtype: int64


In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
X = customer_data[['TotalSpent', 'PurchaseFrequency', 'TotalItems']].values
y = customer_data['Purchased'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train = torch.FloatTensor(X_train).to('cuda')
X_test = torch.FloatTensor(X_test).to('cuda')
y_train = torch.FloatTensor(y_train).to('cuda').reshape(-1, 1)
y_test = torch.FloatTensor(y_test).to('cuda').reshape(-1, 1)
print("New training data shape:", X_train.shape)
print("New test data shape:", X_test.shape)

New training data shape: torch.Size([3871, 3])
New test data shape: torch.Size([968, 3])


In [31]:
import torch
import torch.nn as nn
class CustomerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)
model = CustomerNet().to('cuda')
print(model)

CustomerNet(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [32]:
import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 10, Loss: 0.7029
Epoch 20, Loss: 0.6812
Epoch 30, Loss: 0.6641
Epoch 40, Loss: 0.6503
Epoch 50, Loss: 0.6381
Epoch 60, Loss: 0.6268
Epoch 70, Loss: 0.6165
Epoch 80, Loss: 0.6066
Epoch 90, Loss: 0.5965
Epoch 100, Loss: 0.5859


In [33]:
with torch.no_grad():
    model.eval()
    predictions = model(X_test)
    predictions = (predictions > 0.5).float()
    accuracy = (predictions == y_test).float().mean()
    print(f"Test Accuracy: {accuracy.item():.4f}")

Test Accuracy: 0.9008


In [34]:
import pandas as pd
import numpy as np
customer_data = customer_data[customer_data['CustomerID'] < 999000]  # Reset to original 4,339
non_purchasers = pd.DataFrame({
    'CustomerID': range(999000, 1000000),  # 1,000 non-purchasers
    'TotalSpent': np.random.uniform(0, 50, 1000),
    'PurchaseFrequency': np.ones(1000),
    'TotalItems': np.random.randint(1, 5, 1000),
    'Purchased': np.zeros(1000)
})
customer_data = pd.concat([customer_data, non_purchasers], ignore_index=True)
print("Updated customer data size:", customer_data.shape)
print("Purchased distribution:", customer_data['Purchased'].value_counts())

Updated customer data size: (5339, 5)
Purchased distribution: Purchased
1.0    4339
0.0    1000
Name: count, dtype: int64


In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
X = customer_data[['TotalSpent', 'PurchaseFrequency', 'TotalItems']].values
y = customer_data['Purchased'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train = torch.FloatTensor(X_train).to('cuda')
X_test = torch.FloatTensor(X_test).to('cuda')
y_train = torch.FloatTensor(y_train).to('cuda').reshape(-1, 1)
y_test = torch.FloatTensor(y_test).to('cuda').reshape(-1, 1)
print("New training data shape:", X_train.shape)
print("New test data shape:", X_test.shape)

New training data shape: torch.Size([4271, 3])
New test data shape: torch.Size([1068, 3])


In [36]:
import torch
import torch.nn as nn
class CustomerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)
model = CustomerNet().to('cuda')
print(model)


CustomerNet(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [37]:
import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 10, Loss: 0.7698
Epoch 20, Loss: 0.7588
Epoch 30, Loss: 0.7464
Epoch 40, Loss: 0.7279
Epoch 50, Loss: 0.7057
Epoch 60, Loss: 0.6824
Epoch 70, Loss: 0.6586
Epoch 80, Loss: 0.6345
Epoch 90, Loss: 0.6105
Epoch 100, Loss: 0.5869


In [38]:
with torch.no_grad():
    model.eval()
    predictions = model(X_test)
    predictions = (predictions > 0.5).float()
    accuracy = (predictions == y_test).float().mean()
    print(f"Test Accuracy: {accuracy.item():.4f}")

Test Accuracy: 0.8174
